### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных


In [11]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [12]:
ratings = pd.read_csv('ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [13]:
movie_info = pd.read_csv('movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [14]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [68]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [16]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [17]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [18]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [19]:
model.fit(user_item_t_csr)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [20]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [22]:
get_similars(1, model)

['0    Toy Story (1995)',
 '16    Sense and Sensibility (1995)',
 '486    Malice (1993)',
 '863    Killer: A Journal of Murder (1995)',
 '2051    Needful Things (1993)',
 '3411    Lucas (1986)',
 '1988    Incredible Journey, The (1963)',
 '487    Man Without a Face, The (1993)',
 '3405    Retroactive (1997)',
 '952    Angel and the Badman (1947)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [23]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [24]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [25]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [26]:
get_recommendations(4, model)

['3458    Predator (1987)',
 '1079    Glengarry Glen Ross (1992)',
 '60    Eye for an Eye (1996)',
 '2054    All Dogs Go to Heaven (1989)',
 '2020    Rescuers Down Under, The (1990)',
 '515    Robocop 3 (1993)',
 '1672    Titanic (1997)',
 '1829    Land Girls, The (1998)',
 "3268    I'll Never Forget What's 'is Name (1967)",
 '1462    Sixth Man, The (1997)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [66]:
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
import math

def rmse(u, m):
    pred = (u @ m)[ratings["user_id"], ratings["movie_id"]]
    return metrics.mean_squared_error(ratings["rating"], pred, squared=True)

def rmse_ALS(u, m):
    pred = (u @ m)[implcit_ratings["user_id"], implicit_ratings["movie_id"]]
    real = user_item_csr[implicit_ratings["user_id"], implicit_ratings["movie_id"]]
    real = np.squeeze(np.asarray(real))
    return metrics.mean_squared_error(real, pred, squared=True)

def get_simil1(m, m_id):
    print('Film', movie_info[movie_info['movie_id'] == m_id][['name', 'category']].to_string())
    similarity = np.dot(m[:,m_id], m) / np.sqrt((m * m).sum(axis=0)) / np.sqrt((m[:,m_id] ** 2).sum())
    recomend = np.argsort(similarity)[::-1]
    for i in recomend[:10]:
        print(movie_info[movie_info['movie_id'] == i][['name', 'category']].values)
    return

def get_simil2(m, m_id):
    print('Film', movie_info[movie_info['movie_id'] == m_id][['name', 'category']].to_string())
    movies = m.T
    matr1 = movies[m_id]
    similarity = cosine_similarity(np.expand_dims(matr1, axis=0), movies)[0]
    similarity = np.argsort(similarity)[::-1]
    for i in similarity[:10]:
        print(movie_info[movie_info['movie_id'] == i][['name', 'category']].values)
    return

In [ ]:
import random
import math
from tqdm import tqdm
from sklearn import metrics

users = ratings["user_id"]
movies = ratings["movie_id"]
movie_ratings = ratings["rating"]

explicit = sp.coo_matrix((movie_ratings, (users, movies))).tocsr()
u_amount = explicit.shape[0]
m_amount = explicit.shape[1]
k = 64

u = np.random.uniform(0, 1 / math.sqrt(k), (u_amount, k))
m = np.random.uniform(0, 1 / math.sqrt(k), (k, m_amount))
u_bias = np.random.uniform(0, 1 / math.sqrt(k), (u_amount, 1))
m_bias = np.random.uniform(0, 1 / math.sqrt(k), (1, m_amount))

epochs = 25000000
l = 0.01
r = 0.001

size = ratings.shape[0]
indeces = list()
for i in range(size):
  indeces.append(i)
losses = list()

for epoch in range(epochs):
    ind = indeces[epoch % len(ratings)]
    i = ratings.loc[ind, "user_id"]
    j = ratings.loc[ind, "movie_id"]
    rat = ratings.loc[ind, 'rating']
    prevW = u[i, :]
    diff = np.dot(u[i, :],m[:, j]) - explicit[i, j] # + u_bias[i, :] + m_bias[:, j]
    u[i, :] -= l * diff * m[:, j].T
    m[:, j] -= l * diff * prevW.T
    u_bias[i, :] -= l * (diff + r * u_bias[i, :])
    m_bias[:, j] -= l * (diff + r * m_bias[:, j])
    if (ind % 100000 == 99999):
        print(rmse(u, m))

8.432945284168774
7.576435073145115
7.137204663624338
7.055357479035439
7.216751003196264
7.418044476568979
7.61608790573828
7.271679339499879
7.500806021104189
7.8123554789562855
2.557866393536851
1.627972593128536
1.2665872215698004
1.1047932413622308
1.0479926972085687
1.0193288384193169
0.9987845553237977
0.9586089532081448
0.9203480126041145
0.9000085032301609
0.8727920870030716
0.8640671587855479
0.855984584293257
0.8510387114135136
0.8408799755269503
0.8384139177381226
0.8323141491251093
0.823331974415349
0.8176231560775574
0.813628903561187
0.8047392574954968
0.799673345552522
0.7956764166014711
0.7903197630528052
0.783323465887938
0.7799773145785093
0.7750937887580212
0.7686568943109061
0.7635451187583087
0.7592980045170268
0.7515584279625463
0.7461239040903677
0.742442890406596
0.7367303772259025
0.7291587719993491
0.7248524142065988
0.7202658274397357
0.7134594513277355
0.7078289648574471
0.7025785277326327
0.6951885155179705
0.6892283479898039
0.6848828485610179
0.679742875

In [60]:
get_simil1(m, 1) 
get_simil2(m, 10)

Film                name                     category
0  Toy Story (1995)  Animation|Children's|Comedy
[['Toy Story (1995)' "Animation|Children's|Comedy"]]
[['Terminator 2: Judgment Day (1991)' 'Action|Sci-Fi|Thriller']]
[['Back to the Future (1985)' 'Comedy|Sci-Fi']]
[['Being John Malkovich (1999)' 'Comedy']]
[['Silence of the Lambs, The (1991)' 'Drama|Thriller']]
[['X-Men (2000)' 'Action|Sci-Fi']]
[['2001: A Space Odyssey (1968)' 'Drama|Mystery|Sci-Fi|Thriller']]
[['Stand by Me (1986)' 'Adventure|Comedy|Drama']]
[['Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)'
  'Sci-Fi|War']]
[['Indiana Jones and the Last Crusade (1989)' 'Action|Adventure']]
Film                name                   category
9  GoldenEye (1995)  Action|Adventure|Thriller
[['GoldenEye (1995)' 'Action|Adventure|Thriller']]
[['Independence Day (ID4) (1996)' 'Action|Sci-Fi|War']]
[['Rock, The (1996)' 'Action|Adventure|Thriller']]
[['Doctor Zhivago (1965)' 'Drama|Romance|War']]
[['Manchuri

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [70]:
import random
import math
from tqdm import tqdm

users = ratings["user_id"]
movies = ratings["movie_id"]
movie_ratings = ratings["rating"]

explicit = sp.coo_matrix((movie_ratings, (users, movies))).tocsr()
n = explicit.shape[0]
m = explicit.shape[1]
k = 64

u = np.random.uniform(0, 1 / math.sqrt(k), (n, k))
m = np.random.uniform(0, 1 / math.sqrt(k), (k, m)) 

epochs = 1000
l = 0.001

for epoch in range(epochs):
    M_T = m.T
    users = u.T @ u + l * np.eye(u.shape[1])
    movies = M_T.T @ M_T + l * np.eye(M_T.shape[1])
    users = np.linalg.inv(users)
    movies = np.linalg.inv(movies)
    users = (u @ users.T)
    m = (explicit.T @ users).T
    u = explicit @ (M_T @ movies.T)
    print(rmse_ALS(u, m))

TypeError: '(0             1
1             1
2             1
3             1
4             1
           ... 
1000204    6040
1000205    6040
1000206    6040
1000207    6040
1000208    6040
Name: user_id, Length: 1000209, dtype: int64, 0          1193
1           661
2           914
3          3408
4          2355
           ... 
1000204    1091
1000205    1094
1000206     562
1000207    1096
1000208    1097
Name: movie_id, Length: 1000209, dtype: int64)' is an invalid key

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных